In [454]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [455]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/jiwon')

weather = pd.read_csv('해운대 날씨3.csv')
customer = pd.read_csv('해운대 입장객수3.csv')

In [456]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, Conv2D, Flatten, Input, SimpleRNN, GRU
from keras.layers import Bidirectional
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.layers import concatenate, Concatenate
import datetime
import time
import inspect

# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)

customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수'] = customer['방문객수'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)

In [457]:
# 데이터 합치기
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기", "방문객수"]]

In [458]:
# 결측치 평균, 0 채우기
total_data.loc[total_data['평균수온'] != total_data['평균수온'], '평균수온'] = total_data['평균수온'].mean()
total_data.loc[total_data['평균풍속'] != total_data['평균풍속'], '평균풍속'] = total_data['평균풍속'].mean()
total_data.loc[total_data['평균기압'] != total_data['평균기압'], '평균기압'] = total_data['평균기압'].mean()
total_data.loc[total_data['평균최대파고'] != total_data['평균최대파고'], '평균최대파고'] = total_data['평균최대파고'].mean()
total_data.loc[total_data['평균파주기'] != total_data['평균파주기'], '평균파주기'] = total_data['평균파주기'].mean()
total_data.loc[total_data['방문객수'] != total_data['방문객수'], '방문객수'] = total_data['방문객수'].mean()


## 날씨_예측

In [459]:
# input data
weather_input = total_data[['강수_관측값',"기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기"]]
weather_output = total_data[['강수_관측값','기온']]

In [460]:
# train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input, weather_output, test_size=0.2, random_state=42)

In [463]:
# minmax scaler
min_scaler = MinMaxScaler()
train_x_scaled = min_scaler.fit_transform(train_x)
test_x_scaled = min_scaler.transform(test_x)

In [464]:
train_x_scaled.shape,test_x_scaled.shape #((1175, 9), (294, 9))

((1175, 9), (294, 9))

In [465]:
train_x = train_x_scaled.reshape(1175, 3, 3)
test_x = test_x_scaled.reshape(294, 3, 3)

In [466]:
train_x.shape, test_x.shape

((1175, 3, 3), (294, 3, 3))

In [467]:
# 2.모델구성 / 날씨
model=Sequential()
model.add(Conv1D(256,2,input_shape=(3,3)))
model.add(Conv1D(512,2,activation='LeakyReLU'))
model.add(LSTM(units=256,input_shape=(3,3),return_sequences=True))
model.add(Bidirectional(LSTM(128,activation='swish')))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(16,activation='LeakyReLU'))
model.add(Dense(2))
model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_14 (Conv1D)          (None, 2, 256)            1792      
                                                                 
 conv1d_15 (Conv1D)          (None, 1, 512)            262656    
                                                                 
 lstm_46 (LSTM)              (None, 1, 256)            787456    
                                                                 
 bidirectional_25 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_152 (Dense)           (None, 64)                16448     
                                                                 
 dense_153 (Dense)           (None, 32)                2080      
                                                     

In [ ]:
# 3. 컴파일 훈련 / 날씨
earlyStopping = EarlyStopping(monitor='val_loss', patience=300, mode='min', verbose=1,restore_best_weights=True)
model.compile(loss='mean_squared_error', optimizer='adam')
hist = model.fit(train_x, train_y, epochs=3000, batch_size=512, 
                validation_split=0.2,verbose=2, callbacks=[earlyStopping])

Epoch 1/3000
2/2 - 6s - loss: 645.1669 - val_loss: 551.5250 - 6s/epoch - 3s/step
Epoch 2/3000
2/2 - 1s - loss: 644.0814 - val_loss: 549.4858 - 736ms/epoch - 368ms/step
Epoch 3/3000
2/2 - 1s - loss: 641.5360 - val_loss: 544.6595 - 731ms/epoch - 365ms/step
Epoch 4/3000
2/2 - 1s - loss: 635.6997 - val_loss: 533.5694 - 737ms/epoch - 368ms/step
Epoch 5/3000
2/2 - 1s - loss: 622.2798 - val_loss: 509.1731 - 744ms/epoch - 372ms/step
Epoch 6/3000
2/2 - 1s - loss: 593.2922 - val_loss: 458.6962 - 738ms/epoch - 369ms/step
Epoch 7/3000
2/2 - 1s - loss: 535.1579 - val_loss: 370.7709 - 729ms/epoch - 365ms/step
Epoch 8/3000
2/2 - 1s - loss: 440.8961 - val_loss: 260.0220 - 737ms/epoch - 368ms/step
Epoch 9/3000
2/2 - 1s - loss: 334.7254 - val_loss: 190.8468 - 729ms/epoch - 364ms/step
Epoch 10/3000
2/2 - 1s - loss: 290.3137 - val_loss: 219.7378 - 729ms/epoch - 365ms/step
Epoch 11/3000
2/2 - 1s - loss: 323.1902 - val_loss: 229.0505 - 732ms/epoch - 366ms/step
Epoch 12/3000
2/2 - 1s - loss: 319.1065 - val_l

In [453]:
# 4. 평가, 예측 / 날씨
loss=model.evaluate(test_x,test_y)
pred_y=model.predict(test_x)

print('loss: ',loss)
print('예상 강수량, 기온 : ', pred_y[-1:])

10/10 [==============================] - 0s 8ms/step - loss: 0.2668
loss:  0.26683858036994934
예상 강수량, 기온 :  [[ 0.366534 22.205046]]



LSTM-Bidirectional-Dense
loss:  3.471569776535034
예상 강수량, 기온 :  [[-0.35264927 22.618633  ]]
소요시간 : 7분

SImpleRnn-Bidirectional-Dense
loss:  0.4687313735485077
예상 강수량, 기온 :  [[ 0.12988004 22.5215    ]]
소요시간 : 2분

GRU-Bidirectional-Dense
loss:  0.37580740451812744
예상 강수량, 기온 :  [[ 0.21724612 22.54261   ]]
소요시간 : 6분

Conv1D-Flatten-Dense
loss:  0.06613653898239136
예상 강수량, 기온 :  [[ 0.22584006 22.31144   ]]
소요시간 : 3분

Conv2D-Flatten-Dense
loss:  0.15781544148921967
예상 강수량, 기온 :  [[ 0.24143335 22.692911  ]]
소요시간 : 5분 49초

Conv1D-LSTM-Dense
loss:  258.5815734863281
예상 강수량, 기온 :  [[ 7.076978 26.647423]]
소요시간 : 4분

Dense 
loss:  7.113884925842285
예상 강수량, 기온 :  [[-0.55168146 20.347754  ]]
소요시간 : 48초






relu/swish/relu/swish/relu
loss:  0.8445999622344971
예상 강수량, 기온 :  [[2.0527927e-02 2.2224108e+01]]

all swish
loss:  0.7943930625915527
예상 강수량, 기온 :  [[ 0.15093838 22.121     ]]

all relu
loss:  0.710697591304779
예상 강수량, 기온 :  [[ 0.39223668 22.001013  ]]

all tanh
loss:  339.7808837890625
예상 강수량, 기온 :  [[ 5.6117105 13.591567 ]]

relu/tanh/relu/tanh/relu
loss:  258.8485412597656
예상 강수량, 기온 :  [[ 6.843943 26.562056]]

all elu
loss:  0.30292296409606934
예상 강수량, 기온 :  [[-2.2086855e-02  2.2125095e+01]

relu/elu/relu/elu/relu
loss:  0.30278292298316956
예상 강수량, 기온 :  [[-0.02803025 22.131666  ]]

all LeakyReLU
loss:  0.2869543433189392
예상 강수량, 기온 :  [[ 0.14146769 22.246378  ]]

relu/LeakyReLU/relu/LeakyReLU/relu
loss:  0.6710501313209534
예상 강수량, 기온 :  [[-0.05687914 22.263391  ]]

all PReLU
loss:  3.4739739894866943
예상 강수량, 기온 :  [[ 0.85006523 22.138683  ]]

PReLU/relu/PReLU/relu/PReLU
loss:  0.959674060344696
예상 강수량, 기온 :  [[ 0.1163625 22.16049  ]]





In [ ]:
# 2.모델구성 / 날씨 
model=Sequential()
model.add(LSTM(units=256,input_shape=(3,3),return_sequences=True))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(256,activation='swish'))
model.add(Dense(128,activation='swish'))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(16,activation='swish'))
model.add(Dense(2))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 3, 256)            266240    
                                                                 
 bidirectional_3 (Bidirectio  (None, 1024)             3149824   
 nal)                                                            
                                                                 
 dense_16 (Dense)            (None, 256)               262400    
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                      

In [ ]:
# 2.모델구성 / 날씨
model=Sequential()
model.add(Conv1D(256,2,input_shape=(3,3)))
model.add(Conv1D(512,2,activation='swish'))
model.add(Flatten())
model.add(Dense(256,activation='swish'))
model.add(Dense(128,activation='swish'))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(16,activation='swish'))
model.add(Dense(2))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2, 256)            1792      
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 512)            262656    
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                                 
 dense_23 (Dense)            (None, 128)               32896     
                                                                 
 dense_24 (Dense)            (None, 64)                8256      
                                                                 
 dense_25 (Dense)            (None, 32)               

In [ ]:
# 2.모델구성 / 날씨
model=Sequential()
model.add(Conv2D(filters=256,kernel_size=(2,2),input_shape=(3,3,1)))
model.add(Conv2D(512,(2,2),activation='swish'))
model.add(Flatten())
model.add(Dense(256,activation='swish'))
model.add(Dense(128,activation='swish'))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(16,activation='swish'))
model.add(Dense(2))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2, 2, 256)         1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 1, 512)         524800    
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_28 (Dense)            (None, 256)               131328    
                                                                 
 dense_29 (Dense)            (None, 128)               32896     
                                                                 
 dense_30 (Dense)            (None, 64)                8256      
                                                                 
 dense_31 (Dense)            (None, 32)               

In [ ]:
# 2.모델구성 / 날씨
model=Sequential()
model.add(Conv1D(256,2,input_shape=(3,3)))
model.add(Conv1D(512,2,activation='swish'))
model.add(LSTM(units=256,input_shape=(3,3),return_sequences=True))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(16,activation='swish'))
model.add(Dense(2))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 2, 256)            1792      
                                                                 
 conv1d_7 (Conv1D)           (None, 1, 512)            262656    
                                                                 
 lstm_10 (LSTM)              (None, 1, 256)            787456    
                                                                 
 bidirectional_5 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense_38 (Dense)            (None, 64)                16448     
                                                                 
 dense_39 (Dense)            (None, 32)                2080      
                                                      

In [ ]:
# 2.모델구성 / 날씨
model=Sequential()
model.add(Dense(256,input_shape=(9,)))
model.add(Dense(512,activation='swish'))
model.add(Dense(128,activation='swish'))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(16,activation='swish'))
model.add(Dense(2))
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_36 (LSTM)              (None, 3, 256)            266240    
                                                                 
 bidirectional_18 (Bidirecti  (None, 1024)             3149824   
 onal)                                                           
                                                                 
 dense_109 (Dense)           (None, 256)               262656    
                                                                 
 dense_110 (Dense)           (None, 128)               33024     
                                                                 
 dense_111 (Dense)           (None, 64)                8320      
                                                                 
 dense_112 (Dense)           (None, 32)                2112      
                                                     